In [45]:
import pandas as pd
import datetime as datetime

#df1 = pd.read_csv('PATIENTS.csv')
#df2 = pd.read_csv('ADMISSIONS.csv')
#merged_df = pd.merge(df2, df1[['SUBJECT_ID', 'ADMTIME']], on='SUBJECT_ID', how='left')
#merged_df = pd.merge(df2[['SUBJECT_ID', 'HADM_ID', 'ADMITTIME']], df1[['SUBJECT_ID', 'DOB']], on='SUBJECT_ID', how='left')
#df1 = pd.read_csv('PATIENTS.csv', parse_dates=['DOB'], date_parser=lambda x: pd.datetime.strptime(x, '%Y-%m-%d %H:%M:%S').date())
df1 = pd.read_csv('PATIENTS.csv', parse_dates=['DOB'], date_parser=lambda x: pd.datetime.strptime(x, '%Y-%m-%d %H:%M:%S').date())
df2 = pd.read_csv('ADMISSIONS.csv', parse_dates=['ADMITTIME'], date_parser=lambda x: pd.datetime.strptime(x, '%Y-%m-%d %H:%M:%S').date())
merged_df = pd.merge(df2[['SUBJECT_ID', 'HADM_ID', 'ADMITTIME']], df1[['SUBJECT_ID', 'DOB']], on='SUBJECT_ID', how='left')
df3 = pd.read_csv('DIAGNOSES_ICD.csv')

/var/folders/0r/9nn49j113zb2wny_yypjxrch0000gn/T/ipykernel_78535/14555442.py:9: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  df1 = pd.read_csv('PATIENTS.csv', parse_dates=['DOB'], date_parser=lambda x: pd.datetime.strptime(x, '%Y-%m-%d %H:%M:%S').date())
/var/folders/0r/9nn49j113zb2wny_yypjxrch0000gn/T/ipykernel_78535/14555442.py:10: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  df2 = pd.read_csv('ADMISSIONS.csv', parse_dates=['ADMITTIME'], date_parser=lambda x: pd.datetime.strptime(x, '%Y-%m-%d %H:%M:%S').date())


In [46]:
# Calculation of age
def calculate_age(dob, current_date):
    # Convert the date strings to datetime objects
    #dob = datetime.datetime.strptime(dob, "%Y-%m-%d")
    #current_date = datetime.datetime.strptime(current_date, "%Y-%m-%d")

    # Calculate the age in years
    age = current_date.year - dob.year

    # Adjust the age if the birthday hasn't occurred yet this year
    if (current_date.month, current_date.day) < (dob.month, dob.day):
        age -= 1

    return age

# Example usage
#date_of_birth = "1990-5-12"
#today = "2024-04-09"

date_of_birth = merged_df['DOB'].iloc[0].date()
today = merged_df['ADMITTIME'].iloc[0].date()

D_O_B=date_of_birth.strftime('%Y-%m-%d')
my_today=today.strftime('%Y-%m-%d')
age = calculate_age(date_of_birth, today)
print(f"Age: {age}")

Age: 64


In [47]:
#Calculate the age and add "age" column
merged_df['age'] = merged_df.apply(lambda row: calculate_age(row['DOB'], row['ADMITTIME']), axis=1)

In [48]:
merged_df.head()

,SUBJECT_ID,HADM_ID,ADMITTIME,DOB,age
0,22,165315,2196-04-09,2131-05-07,64
1,23,152223,2153-09-03,2082-07-17,71
2,23,124321,2157-10-18,2082-07-17,75
3,24,161859,2139-06-06,2100-05-31,39
4,25,129635,2160-11-02,2101-11-21,58


In [51]:
# add ICD9_code

# Group df3 by 'pat_id' and aggregate 'medical_code' into a list
df3_grouped = df3.groupby('SUBJECT_ID')['ICD9_CODE'].agg(list).reset_index()

# Rename the aggregated column to 'Med_code'
#df2_grouped.rename(columns={'medical_code': 'Med_code'}, inplace=True)

# Merge df1 and df2_grouped on 'pat_id'
df_merged = pd.merge(merged_df, df3_grouped, on='SUBJECT_ID', how='left')


In [52]:
df_merged.head()

,SUBJECT_ID,HADM_ID,ADMITTIME,DOB,age,ICD9_CODE
0,22,165315,2196-04-09,2131-05-07,64,"[9678, 9693, E9502, E9503, 3488, 29620, 4019]"
1,23,152223,2153-09-03,2082-07-17,71,"[2252, 3485, 78039, 4241, 4019, 2720, 2724, V4..."
2,23,124321,2157-10-18,2082-07-17,75,"[2252, 3485, 78039, 4241, 4019, 2720, 2724, V4..."
3,24,161859,2139-06-06,2100-05-31,39,"[41041, 41401, 53081, 25000]"
4,25,129635,2160-11-02,2101-11-21,58,"[41071, 25011, 41401, 4019]"


In [54]:
selected_columns = ['SUBJECT_ID','ICD9_CODE','age']
df_merged=df_merged[selected_columns]

In [55]:
df_merged.head()

,SUBJECT_ID,ICD9_CODE,age
0,22,"[9678, 9693, E9502, E9503, 3488, 29620, 4019]",64
1,23,"[2252, 3485, 78039, 4241, 4019, 2720, 2724, V4...",71
2,23,"[2252, 3485, 78039, 4241, 4019, 2720, 2724, V4...",75
3,24,"[41041, 41401, 53081, 25000]",39
4,25,"[41071, 25011, 41401, 4019]",58


In [56]:
# Write DataFrame to a Parquet file
df_merged.to_parquet('data.parquet')